In [7]:
#!pip install gym==0.17.3
#!pip install matplotlib
#!pip install tensorflow

In [ ]:
import random
import gym
import numpy as np
from collections import deque
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

'''
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # FATAL only

import tensorflow as tf
tf.get_logger().setLevel('ERROR')

'''


In [ ]:


NUM_ACTIONS = 11#BIN NUMBER IS A PARAM N SHLD BE TUNED
action_bins = np.linspace(-2, 2, NUM_ACTIONS)  # E.g. [-2.0, -1.6, ..., 2.0]


In [ ]:
#qns to ask:
#the action bins are set to -2 to 2, so the action space is continuous, but the action space in the environment is discrete, so how do i map the action bins to the discrete action space?

#the parameters like gamma, epsilon, epsilon_min, epsilon_decay, learning_rate, alpha, etc. are they all correct?

In [17]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=100000)
        self.gamma = 0.99           # Discount factor -----------------ASK CHER
        self.epsilon = 1.0          # Exploration rate 
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.001

        self.model = self._build_model()

    def _build_model(self):
        model = Sequential()
        model.add(Dense(64, input_dim=self.state_size, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))  # Q-values for each discrete action
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state[np.newaxis, :], verbose=0)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target += self.gamma * np.amax(self.model.predict(next_state[np.newaxis, :], verbose=0)[0])
            target_f = self.model.predict(state[np.newaxis, :], verbose=0)
            target_f[0][action] = target
            self.model.fit(state[np.newaxis, :], target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [ ]:
env = gym.make('Pendulum-v0')  # gym 0.17.3
state_size = env.observation_space.shape[0]
action_size = NUM_ACTIONS  # number of discrete actions

agent = DQNAgent(state_size, action_size)
episodes = 500
#do i need to set a limit-set the epoch-cos if it hit the max reward it will auto stop-so shldnt i jus let it run infiately and let it run ntil it get the max reward?
batch_size = 64

for e in range(episodes):
    state = env.reset()
    
    total_reward = 0
    for time in range(200):
        action_idx = agent.act(state)
        action = [action_bins[action_idx]]  # env expects an array
        next_state, reward, done, _ = env.step(action)
        agent.remember(state, action_idx, reward, next_state, done)
        state = next_state
        total_reward += reward
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)
        if done:
            break
    print(f"Episode {e+1}: Reward = {total_reward}")
  


KeyboardInterrupt: 